# Exploratory Data Analysis (EDA) and Extract Transform Load (ETL)

In [1]:
import html

from ast import literal_eval
from constants import Paths
from dataclasses import dataclass
from functools import partial
from IPython.display import display, display_html, display_markdown
from pandas import DataFrame, read_csv
from pathlib import Path
from pprint import pformat

display_html = partial(display_html, raw = True)

display_markdown = partial(display_markdown, raw = True)

def display_python_structure(value) -> None:
    print(pformat(value, 4))

def display_top_rows(df: DataFrame) -> None:
    display_markdown('#### Top Rows')
    display(df.head(10))
    display_markdown('**Dataframe shape:** {:,} rows × {:,} columns'.format(*df.shape))

def ensure_parent_exists(path: str) -> None:
    Path(path).parent.mkdir(parents = True, exist_ok = True)

## USDA FoodData Central Dataset

In [2]:
@dataclass(frozen = True, slots = True)
class NutrientInfo:
    name: str
    dv_factor: float

def read_nutrient_data():
    return read_csv(Paths.NUTRIENT_DATASET, index_col = 'id')

def read_food_data():
    return read_csv(Paths.FOOD_DATASET, index_col = 'fdc_id')

def read_food_nutrient_data():
    return read_csv(Paths.FOOD_NUTRIENT_DATASET, index_col = 'id')

### Nutrients

In [3]:
nutrient_df: DataFrame = read_nutrient_data()

display_top_rows(nutrient_df)

#### Top Rows

,name,unit_name,nutrient_nbr,rank
id,,,,
2047,Energy (Atwater General Factors),KCAL,957.0,280.0
2048,Energy (Atwater Specific Factors),KCAL,958.0,290.0
1001,Solids,G,201.0,200.0
1002,Nitrogen,G,202.0,500.0
1003,Protein,G,203.0,600.0
1004,Total lipid (fat),G,204.0,800.0
1005,"Carbohydrate, by difference",G,205.0,1110.0
1006,"Fiber, crude (DO NOT USE - Archived)",G,206.0,999999.0
1007,Ash,G,207.0,1000.0


**Dataframe shape:** 477 rows × 4 columns

Hmm… it’s got a lot of extra data we do not need. We need to be selective.

### Foods

In [4]:
food_df: DataFrame = read_food_data()

display_top_rows(food_df)

#### Top Rows

,data_type,description,food_category_id,publication_date
fdc_id,,,,
1105904,branded_food,WESSON Vegetable Oil 1 GAL,Oils Edible,2020-11-13
1105905,branded_food,SWANSON BROTH BEEF,Herbs/Spices/Extracts,2020-11-13
1105906,branded_food,CAMPBELL'S SLOW KETTLE SOUP CLAM CHOWDER,Prepared Soups,2020-11-13
1105907,branded_food,CAMPBELL'S SLOW KETTLE SOUP CHEESE BROCCOLI,Prepared Soups,2020-11-13
1105898,experimental_food,Discrepancy between the Atwater factor predict...,NaN,2020-10-30
1105899,experimental_food,Food processing and structure impact the metab...,NaN,2020-10-30
1105900,experimental_food,Metabolizable Energy from Cashew Nuts is Less ...,NaN,2020-10-30
1105901,experimental_food,Measured energy value of pistachios in the hum...,NaN,2020-10-30
1105902,experimental_food,Walnuts Consumed by Healthy Adults Provide Les...,NaN,2020-10-30


**Dataframe shape:** 2,059,641 rows × 4 columns

OK. We got our foods. Now… what foods have what?

### Foods × Nutrients

In [5]:
food_nutrient_df: DataFrame = read_food_nutrient_data()

display_top_rows(food_nutrient_df)

C:\Users\Kevin Li\AppData\Local\Temp\ipykernel_5220\1771124419.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(Paths.FOOD_NUTRIENT_DATASET, index_col = 'id')


#### Top Rows

,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired,percent_daily_value
id,,,,,,,,,,,,
13706927,1105904,1257,0.00,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13706930,1105904,1293,53.33,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13706926,1105904,1253,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13706921,1105904,1092,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13706916,1105904,1008,867.00,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13706920,1105904,1089,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13706925,1105904,1162,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
13706928,1105904,1258,13.33,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0
13706919,1105904,1087,0.00,NaN,75.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


**Dataframe shape:** 26,770,242 rows × 12 columns

Let’s join them and see if we can spot any patterns in the resulting data.

In [6]:
nutrition_fact_df: DataFrame = food_nutrient_df[['fdc_id', 'nutrient_id', 'amount']] \
                              .join(food_df[['description']], 'fdc_id', 'inner') \
                              .join(nutrient_df[['name', 'unit_name']], 'nutrient_id', 'inner') \
                              .pipe(lambda df: df.drop_duplicates(['fdc_id', 'nutrient_id'])) \
                              .pipe(lambda df: df.assign(nutrient_value = df.apply(lambda row: f'{row.amount:,} {row.unit_name.lower()}', axis = 1))) \
                              .pivot(index = ['fdc_id', 'description'], columns = ['nutrient_id', 'name'], values = 'nutrient_value') \
                              .dropna(how = 'all')

display_top_rows(nutrition_fact_df)

#### Top Rows

,nutrient_id,1257,1293,1253,1092,1008,1089,1162,1258,1087,1292,...,2063,1041,1044,2038,2065,1043,1068,2067,2068,1149
,name,"Fatty acids, total trans","Fatty acids, total polyunsaturated",Cholesterol,"Potassium, K",Energy,"Iron, Fe","Vitamin C, total ascorbic acid","Fatty acids, total saturated","Calcium, Ca","Fatty acids, total monounsaturated",...,Verbascose,Oxalic acid,Quinic acid,High Molecular Weight Dietary Fiber (HMWDF),Low Molecular Weight Dietary Fiber (LMWDF),Pyruvic acid,Beta-glucans,Vitamin A,Vitamin E,"Salt, NaCl"
fdc_id,description,,,,,,,,,,,,,,,,,,,,,
167512,"Pillsbury Golden Layer Buttermilk Biscuits, Artificial Flavor, refrigerated dough",4.412 g,NaN,0.0 mg,NaN,307.0 kcal,2.12 mg,NaN,2.941 g,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167513,"Pillsbury, Cinnamon Rolls with Icing, refrigerated dough",4.29 g,NaN,0.0 mg,NaN,330.0 kcal,1.93 mg,0.1 mg,3.25 g,28.0 mg,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167514,"Kraft Foods, Shake N Bake Original Recipe, Coating for Pork, dry",NaN,NaN,NaN,NaN,377.0 kcal,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167515,"George Weston Bakeries, Thomas English Muffins",0.082 g,0.92 g,NaN,NaN,232.0 kcal,1.4 mg,NaN,0.308 g,180.0 mg,0.303 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167516,"Waffles, buttermilk, frozen, ready-to-heat",NaN,1.445 g,15.0 mg,126.0 mg,273.0 kcal,6.04 mg,0.0 mg,1.898 g,279.0 mg,4.53 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167517,"Waffle, buttermilk, frozen, ready-to-heat, toasted",NaN,1.502 g,13.0 mg,138.0 mg,309.0 kcal,6.59 mg,0.0 mg,2.275 g,299.0 mg,5.292 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167518,"Waffle, buttermilk, frozen, ready-to-heat, microwaved",NaN,1.615 g,16.0 mg,110.0 mg,289.0 kcal,6.53 mg,0.0 mg,2.057 g,125.0 mg,4.864 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167519,"Waffle, plain, frozen, ready-to-heat, microwave",NaN,2.24 g,16.0 mg,148.0 mg,298.0 kcal,5.81 mg,NaN,1.58 g,197.0 mg,5.195 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Dataframe shape:** 1,944,807 rows × 268 columns

Whoah! That’s a lot of missing values. Can we just drop the foods for which we have very little nutrition data on? Let’s make the criteria be “missing a few key nutrition facts.”

In [7]:
key_nutrients: list[str] = nutrition_fact_df.describe() \
                                            .transpose() \
                                            .sort_values('count', ascending = False) \
                                            .head(12) \
                                            .index \
                                            .to_list()

display_markdown('**Important Values**')
display_python_structure(key_nutrients)

**Important Values**

[   (1003, 'Protein'),
    (1005, 'Carbohydrate, by difference'),
    (1093, 'Sodium, Na'),
    (1008, 'Energy'),
    (1004, 'Total lipid (fat)'),
    (2000, 'Total Sugars'),
    (1258, 'Fatty acids, total saturated'),
    (1253, 'Cholesterol'),
    (1079, 'Fiber, total dietary'),
    (1257, 'Fatty acids, total trans'),
    (1087, 'Calcium, Ca'),
    (1089, 'Iron, Fe')]


In [8]:
nutrition_fact_df: DataFrame = nutrition_fact_df.dropna(subset = key_nutrients)

display_top_rows(nutrition_fact_df)

#### Top Rows

,nutrient_id,1257,1293,1253,1092,1008,1089,1162,1258,1087,1292,...,2063,1041,1044,2038,2065,1043,1068,2067,2068,1149
,name,"Fatty acids, total trans","Fatty acids, total polyunsaturated",Cholesterol,"Potassium, K",Energy,"Iron, Fe","Vitamin C, total ascorbic acid","Fatty acids, total saturated","Calcium, Ca","Fatty acids, total monounsaturated",...,Verbascose,Oxalic acid,Quinic acid,High Molecular Weight Dietary Fiber (HMWDF),Low Molecular Weight Dietary Fiber (LMWDF),Pyruvic acid,Beta-glucans,Vitamin A,Vitamin E,"Salt, NaCl"
fdc_id,description,,,,,,,,,,,,,,,,,,,,,
167513,"Pillsbury, Cinnamon Rolls with Icing, refrigerated dough",4.29 g,NaN,0.0 mg,NaN,330.0 kcal,1.93 mg,0.1 mg,3.25 g,28.0 mg,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167521,"Pie Crust, Cookie-type, Chocolate, Ready Crust",0.0 g,2.035 g,0.0 mg,187.0 mg,484.0 kcal,4.3 mg,0.0 mg,4.725 g,32.0 mg,14.482 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167524,"Waffles, chocolate chip, frozen, ready-to-heat",0.2 g,3.272 g,21.0 mg,74.0 mg,297.0 kcal,6.4 mg,0.0 mg,3.567 g,357.0 mg,2.564 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167526,"Bread, salvadoran sweet cheese (quesadilla salvadorena)",2.027 g,4.428 g,59.0 mg,350.0 mg,374.0 kcal,0.98 mg,0.0 mg,4.612 g,73.0 mg,6.794 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167528,"Pastry, Pastelitos de Guava (guava pastries)",2.853 g,2.835 g,10.0 mg,103.0 mg,379.0 kcal,2.12 mg,NaN,5.535 g,14.0 mg,8.543 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167532,"Bread, white wheat",0.031 g,0.973 g,0.0 mg,127.0 mg,238.0 kcal,4.89 mg,0.0 mg,0.63 g,684.0 mg,0.393 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167533,"Bagels, wheat",0.0 g,0.936 g,0.0 mg,165.0 mg,250.0 kcal,2.76 mg,0.0 mg,0.0 g,20.0 mg,0.29 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167534,"Cream puff, eclair, custard or cream filled, iced",0.0 g,4.173 g,66.0 mg,68.0 mg,334.0 kcal,0.95 mg,0.0 mg,7.055 g,35.0 mg,6.07 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Dataframe shape:** 1,374,521 rows × 268 columns

Let’s do the same to columns which are mostly empty.

In [9]:
spotty_nutrients: list[str] = nutrition_fact_df.describe() \
                                               .transpose() \
                                               .sort_values('count') \
                                               .pipe(lambda df: df[df['count'] < df['count'].quantile(0.85)]) \
                                               .index \
                                               .to_list()

display_markdown('**Unimportant Values**')
display_python_structure(spotty_nutrients)

**Unimportant Values**

[   (1149, 'Salt, NaCl'),
    (1121, 'Lutein'),
    (1159, 'cis-beta-Carotene'),
    (1160, 'cis-Lycopene'),
    (2032, 'Cryptoxanthin, alpha'),
    (1161, 'cis-Lutein/Zeaxanthin'),
    (1330, 'Fatty acids, total trans-dienoic'),
    (1050, 'Carbohydrate, by summation'),
    (2026, 'PUFA 20:2 c'),
    (1063, 'Sugars, Total'),
    (2020, 'PUFA 20:3 c'),
    (2024, 'PUFA 22:5 c'),
    (2010, 'MUFA 17:1 c'),
    (2016, 'PUFA 18:2 c'),
    (2018, 'PUFA 18:3 c'),
    (2023, 'PUFA 20:5c'),
    (2025, 'PUFA 22:6 c'),
    (2022, 'PUFA 20:4c'),
    (2047, 'Energy (Atwater General Factors)'),
    (2048, 'Energy (Atwater Specific Factors)'),
    (1117, 'Phytofluene'),
    (1116, 'Phytoene'),
    (2029, 'trans-Lycopene'),
    (2028, 'trans-beta-Carotene'),
    (1119, 'Zeaxanthin'),
    (2003, 'SFA 5:0'),
    (1335, 'SFA 11:0'),
    (1414, 'PUFA 20:3 n-9'),
    (2012, 'MUFA 20:1 c'),
    (1334, 'PUFA 22:2'),
    (2014, 'MUFA 22:1 n-9'),
    (2019, 'TFA 18:3 t'),
    (2009, 'MUFA 14:1 c'),
    (1197

In [10]:
nutrition_fact_df: DataFrame = nutrition_fact_df.drop(columns = spotty_nutrients)

display_top_rows(nutrition_fact_df)

#### Top Rows

,nutrient_id,1257,1293,1253,1092,1008,1089,1162,1258,1087,1292,...,1100,1095,1158,1082,1084,1086,1185,1098,1114,2067
,name,"Fatty acids, total trans","Fatty acids, total polyunsaturated",Cholesterol,"Potassium, K",Energy,"Iron, Fe","Vitamin C, total ascorbic acid","Fatty acids, total saturated","Calcium, Ca","Fatty acids, total monounsaturated",...,"Iodine, I","Zinc, Zn",Vitamin E,"Fiber, soluble","Fiber, insoluble",Total sugar alcohols,Vitamin K (phylloquinone),"Copper, Cu",Vitamin D (D2 + D3),Vitamin A
fdc_id,description,,,,,,,,,,,,,,,,,,,,,
167513,"Pillsbury, Cinnamon Rolls with Icing, refrigerated dough",4.29 g,NaN,0.0 mg,NaN,330.0 kcal,1.93 mg,0.1 mg,3.25 g,28.0 mg,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167521,"Pie Crust, Cookie-type, Chocolate, Ready Crust",0.0 g,2.035 g,0.0 mg,187.0 mg,484.0 kcal,4.3 mg,0.0 mg,4.725 g,32.0 mg,14.482 g,...,NaN,2.1 mg,NaN,NaN,NaN,NaN,18.2 ug,0.77 mg,0.0 ug,NaN
167524,"Waffles, chocolate chip, frozen, ready-to-heat",0.2 g,3.272 g,21.0 mg,74.0 mg,297.0 kcal,6.4 mg,0.0 mg,3.567 g,357.0 mg,2.564 g,...,NaN,0.4 mg,NaN,NaN,NaN,NaN,9.3 ug,0.221 mg,0.1 ug,NaN
167526,"Bread, salvadoran sweet cheese (quesadilla salvadorena)",2.027 g,4.428 g,59.0 mg,350.0 mg,374.0 kcal,0.98 mg,0.0 mg,4.612 g,73.0 mg,6.794 g,...,NaN,0.8 mg,NaN,NaN,NaN,NaN,11.2 ug,0.03 mg,0.3 ug,NaN
167528,"Pastry, Pastelitos de Guava (guava pastries)",2.853 g,2.835 g,10.0 mg,103.0 mg,379.0 kcal,2.12 mg,NaN,5.535 g,14.0 mg,8.543 g,...,NaN,0.42 mg,NaN,NaN,NaN,NaN,12.5 ug,0.079 mg,NaN,NaN
167532,"Bread, white wheat",0.031 g,0.973 g,0.0 mg,127.0 mg,238.0 kcal,4.89 mg,0.0 mg,0.63 g,684.0 mg,0.393 g,...,NaN,0.95 mg,NaN,NaN,NaN,NaN,7.7 ug,0.149 mg,0.0 ug,NaN
167533,"Bagels, wheat",0.0 g,0.936 g,0.0 mg,165.0 mg,250.0 kcal,2.76 mg,0.0 mg,0.0 g,20.0 mg,0.29 g,...,NaN,1.1 mg,NaN,NaN,NaN,NaN,1.5 ug,0.175 mg,0.0 ug,NaN
167534,"Cream puff, eclair, custard or cream filled, iced",0.0 g,4.173 g,66.0 mg,68.0 mg,334.0 kcal,0.95 mg,0.0 mg,7.055 g,35.0 mg,6.07 g,...,NaN,0.45 mg,NaN,NaN,NaN,NaN,6.7 ug,0.06 mg,0.3 ug,NaN


**Dataframe shape:** 1,374,521 rows × 41 columns

What’s left after the fray?

In [11]:
nutrition_fact_df.columns \
                 .to_frame(index = False) \
                 .set_index('nutrient_id') \
                 .join(nutrient_df.unit_name) \
                 .sort_values('name') \
                 .pipe(display)

,name,unit_name
nutrient_id,,
1087,"Calcium, Ca",MG
1005,"Carbohydrate, by difference",G
1253,Cholesterol,MG
1098,"Copper, Cu",MG
1008,Energy,KCAL
1292,"Fatty acids, total monounsaturated",G
1293,"Fatty acids, total polyunsaturated",G
1258,"Fatty acids, total saturated",G
1257,"Fatty acids, total trans",G


With the analysis above informing us, let’s be very selective about what to keep.

### Extract, Transform, and Load

#### Nutrient Selection

This part requires a bit of additional legwork and manual fine-tuning. Use the findings above as a clue, and research the info you need to fill in.

In [12]:
select_nutrients: dict[int, NutrientInfo] = {  # Nutrient identifier mapped to column name and DV factor
    1004: NutrientInfo('total_fat',       78), # g  of total fat
    1253: NutrientInfo('cholesterol',    300), # mg of cholesterol
    1005: NutrientInfo('carbohydrate',   300), # g  of carbohydrate
    1079: NutrientInfo('dietary_fiber',   28), # g  of dietary fiber
    1003: NutrientInfo('protein',         50), # g  of protein
    1104: NutrientInfo('vitamin_a',     3000), # UI of vitamin A
    1167: NutrientInfo('niacin',          16), # mg of niacin
    1166: NutrientInfo('riboflavin',     1.3), # mg of riboflavin
    1165: NutrientInfo('thiamin',        1.2), # mg of thiamin
    1170: NutrientInfo('pantothenic_acid', 5), # mg of pantothenic acid
    1175: NutrientInfo('vitamin_b6',     1.7), # mg of vitamin B₆
    1177: NutrientInfo('folate',         400), # µg of folate
    1178: NutrientInfo('vitamin_b12',    2.4), # µg of vitamin B₁₂
    1162: NutrientInfo('vitamin_c',       90), # mg of vitamin C
    1110: NutrientInfo('vitamin_d',      800), # IU of vitamin D
    1158: NutrientInfo('vitamin_e',       15), # mg of vitamin E
    1185: NutrientInfo('vitamin_c',      120), # µg of vitamin K
    1087: NutrientInfo('calcium',       1000), # mg of calcium
    1098: NutrientInfo('copper',         900), # µg of copper
    1100: NutrientInfo('iodine',         150), # µg of iodine
    1089: NutrientInfo('iron',            18), # mg of iron
    1090: NutrientInfo('magnesium',      420), # mg of magnesium
    1101: NutrientInfo('manganese',      2.3), # mg of manganese
    1091: NutrientInfo('phosphorus',    1250), # mg of phosphorus
    1092: NutrientInfo('potassium',     3400), # mg of potassium
    1103: NutrientInfo('selenium',        55), # µg of selenium
    1093: NutrientInfo('sodium',        2300), # mg of sodium
    1095: NutrientInfo('zinc',            11)  # mg of zinc
}

#### Save the Table as a Resource for Further Use

Assuming you’ve defined `select_nutrients: dict[int, NutrientInfo]` above, the following extracts, transforms, and saves the nutrition data in one step.

In [13]:
ensure_parent_exists(Paths.NUTRITION_DATA_RESOURCE)
read_food_nutrient_data().drop_duplicates(['fdc_id', 'nutrient_id']) \
                         .pipe(lambda df: df[df.nutrient_id.isin(select_nutrients)]) \
                         .pipe(lambda df: df.assign(dv            = df.apply(lambda row: row.amount / select_nutrients[row.nutrient_id].dv_factor, axis = 1),
                                                    nutrient_name = df.apply(lambda row: select_nutrients[row.nutrient_id].name, axis = 1))) \
                         .get(['fdc_id', 'nutrient_name', 'dv']) \
                         .join(read_food_data().get(['description']), 'fdc_id', 'inner') \
                         .drop_duplicates(['fdc_id', 'description']) \
                         .pivot(index = ['fdc_id', 'description'], columns = 'nutrient_name', values = 'dv') \
                         .to_csv(Paths.NUTRITION_DATA_RESOURCE)
display_html(f'Saved nutrition data to <em>{html.escape(Paths.NUTRITION_DATA_RESOURCE)}</em>.')

C:\Users\Kevin Li\AppData\Local\Temp\ipykernel_5220\1771124419.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(Paths.FOOD_NUTRIENT_DATASET, index_col = 'id')


Saved nutrition data to resources/nutrition.csv .

## Food.com Recipes with Ingredients and Tags Dataset

In [14]:
def read_recipe_term_tag_data():
    return read_csv(Paths.RECIPE_TERM_TAG_DATASET,
                    index_col = 'id',
                    converters = {
                        column: html.unescape # Converts HTML escape sequences back to human-readable characters
                        for column in ('name', 'description')
                    } | {
                        column: lambda value: tuple(map(html.unescape, literal_eval(value)))
                        for column in ('ingredients', 'ingredients_raw_str', 'search_terms', 'steps')
                    } | {
                        'tags': lambda value: frozenset(map(html.unescape, literal_eval(value)))
                    })

In [15]:
recipe_term_tag_df: DataFrame = read_recipe_term_tag_data()

display_top_rows(recipe_term_tag_df)
display_markdown('---')
display_markdown('#### Top Row\n\nLook at this *structured* data. 🙂')
display_python_structure(recipe_term_tag_df.iloc[0].to_dict())

#### Top Rows

,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms
id,,,,,,,,,
96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","(water, grits, salt, cheddar cheese, garlic, o...","(4 cups water, 1 cup uncooked old fas...",1 (155 g),8,"(I a sauce pan, bring water to a boil; slowly ...","(pasta-rice-and-grains, stove-top, presentatio...","(low-carb, diabetic, low-calorie, side, vegeta..."
232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","(onion, red bell pepper, garlic cloves, large ...","(1 medium onion, chopped coarse , 1 med...",1 (366 g),4,"(In a food processor, pulse the onion, red pep...","(course, shrimp, preparation, main-ingredient,...","(dinner, shrimp)"
41090,black-and-white bean salad,,"(white beans, canned black beans, tomatoes, on...","(1 cup canned white beans, rinsed and drai...",1 (807 g),1,"(In a large bowl, combine beans, tomato, onion...","(dinner-party, 3-steps-or-less, american, bean...","(vegan, salad, side, dinner, vegetarian)"
60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"(zucchini, yellow squash, diced tomatoes, onio...","(2 zucchini, sliced , 2 small yello...",1 (244 g),4,(Put all ingredients in the crock pot and cook...,"(3-steps-or-less, vegetables, equipment, prepa...","(italian, vegetarian, side)"
232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"(beef stew meat, flour, salt, allspice, cinnam...","(3 lbs beef stew meat, 3 tablespoons ...",1 (358 g),8,"(Preheat oven to 350°F., Cut beef into 1 inch ...","(course, preparation, main-ingredient, main-di...","(dinner,)"
232050,Hot Sweet Almond Brittle,This is one of our standard holiday gift recip...,"(slivered almonds, cider vinegar, sugar, sugar...","(12 ounces slivered almonds, 1/4 cup ...",1 (832 g),1,(Preheat oven to 375°F Place almonds in singl...,"(low-cholesterol, course, preparation, time-to...","(dessert,)"
232076,Retro Chicken & Chips Casserole,"From Cooking Light Magazine, 11/05. This is co...","(chicken breasts, green onion, red bell pepper...","(4 cups roasted chopped chicken breasts, 1...",1 (85 g),6,"(In large bowl, combine chicken, green onion, ...","(course, preparation, main-ingredient, cassero...","(chicken, dinner, casserole)"
232083,Asparagus Omelette Wraps,"These wraps make a lovely breakfast, light lun...","(eggs, milk, fresh sage, fresh thyme, garlic c...","(8 eggs, 1/2 cup milk, 1 tablespoo...",1 (499 g),4,"(Beat the eggs in a bowl. Add the milk, sage, ...","(course, cuisine, gluten-free, preparation, ma...","(gluten-free, breakfast)"
79222,Potato-Crab Chowder,Soup for the soul!,"(butter, onion, garlic, potatoes, flour, milk,...","(2 tablespoons butter, 1 medium onio...",1 (362 g),6,(Saute onion& garlic in melted butter in large...,"(stove-top, vegetables, low-fat, low-calorie, ...","(low-fat, healthy, low-calorie, low-sodium)"


**Dataframe shape:** 494,963 rows × 9 columns

---

#### Top Row

Look at this *structured* data. 🙂

{   'description': 'We love grits, this is another good way to serve them. A '
                   'great alternative to a baked potato when served with '
                   'grilled steak or chicken. I belive this recipe could be '
                   'made with instant grits.The 2 1/2 hours for refrigeration '
                   'is not include in time. The recipe comes from Tast of '
                   "Home's Light and Tasty.",
    'ingredients': (   'water',
                       'grits',
                       'salt',
                       'cheddar cheese',
                       'garlic',
                       'olive oil'),
    'ingredients_raw_str': (   '4   cups    water',
                               '1   cup   uncooked old fashion grits',
                               '1   teaspoon    salt',
                               '4   ounces   shredded cheddar cheese',
                               '1 -2   clove    garlic, minced ',
                               '1   tablespoo